## Reading Tweets from msca-bdp-tweets bucket

### Using PySpark kernel

In [1]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import StructType

import findspark
findspark.init()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark.sql.functions as sql_fun


In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [4]:
%%time

tweets_df = spark.read.json(path)

22/03/18 22:27:52 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 2.01 s, sys: 538 ms, total: 2.54 s
Wall time: 10min 51s


22/03/18 22:38:35 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Cut df

In [5]:
from pyspark.sql.functions import col
sample_df = tweets_df.select(col("id"),col("id_str"),col("user"),col("place"),col("text"),col("retweeted_status"), col("extended_tweet"), \
                             col("entities"),col("created_at"), col("coordinates"),col("favorite_count"), \
                             col("quote_count"),col("retweet_count"),col("reply_count"),col("is_quote_status"), \
                             col("retweeted"),col("lang"))

In [6]:
import pyspark.sql.functions as sql_fun
sample_df = sample_df.withColumn('text',sql_fun.lower(sample_df.text)).withColumn('retweeted_status_text',sql_fun.lower(sample_df.retweeted_status.text))

In [ ]:
covid_df.limit(5)

id,id_str,user,place,text,retweeted_status,extended_tweet,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,retweeted_status_text,created_at_year,created_at_mon,created_at_day
1469800322834976770,1469800322834976770,"{false, Wed Aug 2...",null,@vlg890 @sandiesw...,null,"{[39, 190], {[], ...","{[], null, [], [{...",Sat Dec 11 22:44:...,null,0,0,0,0,false,false,en,null,2021,12,11
1469800323019468801,1469800323019468801,"{false, Tue May 1...",null,no it’s not that....,null,"{[0, 166], {[], n...","{[], null, [], [{...",Sat Dec 11 22:44:...,null,0,0,0,0,true,false,en,null,2021,12,11
1469800323095146498,1469800323095146498,"{false, Thu Feb 0...",null,rt @ollysmithtrav...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,we will need @ukl...,2021,12,11
1469800323254472709,1469800323254472709,"{false, Fri Nov 0...",null,@iluvchibiis covi...,null,null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,false,en,null,2021,12,11
1469800323564851204,1469800323564851204,"{false, Thu Sep 2...",null,rt @comradevero: ...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,if you’re triple ...,2021,12,11


In [7]:
from pyspark.sql.functions import col,isnan, when, count
col_list = ['retweeted_status']
sample_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+----------------+
|retweeted_status|
+----------------+
|        30331729|
+----------------+



In [9]:
covid_df = sample_df.filter(sample_df.text.contains('covid')|sample_df.retweeted_status_text.contains('covid'))
covid_df.count()

61852096

In [10]:
covid_df = covid_df.withColumn('retweeted', when(covid_df.retweeted_status.isNull(), False).otherwise(True))

In [11]:
#date extraction using regexp
covid_df = covid_df.withColumn("created_at_year", regexp_extract("created_at","20\d\d$",0))
covid_df = covid_df.withColumn("created_at_mon", month(to_date(trim(regexp_extract("created_at","\s\w+",0)),"MMM")))
covid_df = covid_df.withColumn("created_at_day", trim(regexp_extract("created_at","\s\d\d\s",0)))


In [14]:
covid_df = covid_df.withColumn('user_name',col('user.name')) \
                .withColumn('user_scrname',col('user.screen_name')) \
                .withColumn('user_desc',col('user.description')) \
                .withColumn('user_id',col('user.id')) \
                .withColumn('user_loc',col('user.location')) \
                .withColumn('user_url',col('user.url')) \
                .withColumn('user_followers',col('user.followers_count')) \
                .withColumn('user_following',col('user.following')) \
                .withColumn('user_friends_count',col('user.friends_count')) \
                .withColumn('user-verified',col('user.verified')) \
                .withColumn('user_dpimage',col('user.default_profile_image'))


In [12]:
# covid_df = covid_df.drop('user.name', 'user.scrname', 'user.desc', 'user.id', 'user.loc', 'user.url', 'user.followers', 'user.following', 'user.friends_count', 'user.verified', 'user.dpimage')

In [13]:
covid_df.columns

['id',
 'id_str',
 'user',
 'place',
 'text',
 'extended_tweet',
 'entities',
 'created_at',
 'coordinates',
 'favorite_count',
 'quote_count',
 'retweet_count',
 'reply_count',
 'is_quote_status',
 'retweeted',
 'lang',
 'created_at_year',
 'created_at_mon',
 'created_at_day']

In [ ]:
covid_df.limit(10)

id,id_str,user,place,text,retweeted_status,extended_tweet,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,retweeted_status_text,created_at_year,created_at_mon,created_at_day
1469800322834976770,1469800322834976770,"{false, Wed Aug 2...",null,@vlg890 @sandiesw...,null,"{[39, 190], {[], ...","{[], null, [], [{...",Sat Dec 11 22:44:...,null,0,0,0,0,false,false,en,null,2021,12,11
1469800323019468801,1469800323019468801,"{false, Tue May 1...",null,no it’s not that....,null,"{[0, 166], {[], n...","{[], null, [], [{...",Sat Dec 11 22:44:...,null,0,0,0,0,true,false,en,null,2021,12,11
1469800323095146498,1469800323095146498,"{false, Thu Feb 0...",null,rt @ollysmithtrav...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,we will need @ukl...,2021,12,11
1469800323254472709,1469800323254472709,"{false, Fri Nov 0...",null,@iluvchibiis covi...,null,null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,false,en,null,2021,12,11
1469800323564851204,1469800323564851204,"{false, Thu Sep 2...",null,rt @comradevero: ...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,if you’re triple ...,2021,12,11
1469800323573239819,1469800323573239819,"{false, Thu Jul 1...",null,rt @pippacrerar: ...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,exclusive: boris ...,2021,12,11
1469800323787083776,1469800323787083776,"{false, Wed Aug 2...",null,rt @mirrorpolitic...,"{null, null, Sat ...",null,"{[], null, [], [{...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,boris johnson pic...,2021,12,11
1469800323921465345,1469800323921465345,"{false, Wed Mar 0...",null,rt @politicsforal...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,🚨🚨🚨🚨 | breaki...,2021,12,11
1469800323984285702,1469800323984285702,"{false, Wed Oct 0...",null,rt @pippacrerar: ...,"{null, null, Sat ...",null,"{[], null, [], []...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,exclusive: boris ...,2021,12,11
1469800324823060483,1469800324823060483,"{false, Fri Jul 2...",null,rt @deepbluecrypt...,"{null, null, Wed ...",null,"{[], [{{null, nul...",Sat Dec 11 22:44:...,null,0,0,0,0,false,true,en,are these genuine...,2021,12,11


# partition and parquet

In [ ]:
covid_df.write.partitionBy("created_at_year", "created_at_mon", "created_at_day") \
        .mode("overwrite") \
        .parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/covid_tweets4_df.parquet")

22/03/18 23:47:56 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1647641937773_0001_01_000022 on host: hub-msca-bdp-dphub-students-bhadri-sw-50zn.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-03-18 23:47:56.779]Container killed on request. Exit code is 143
[2022-03-18 23:47:56.780]Container exited with a non-zero exit code 143. 
[2022-03-18 23:47:56.784]Killed by external signal
.
22/03/18 23:47:56 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 22 for reason Container from a bad node: container_1647641937773_0001_01_000022 on host: hub-msca-bdp-dphub-students-bhadri-sw-50zn.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-03-18 23:47:56.779]Container killed on request. Exit code is 143
[2022-03-18 23:47:56.780]Container exited with a non-zero exit code 143. 
[2022-03-18 23:47:56.784]Killed by external signal
.
22/03/18 23:47:56 ERROR or

In [ ]:
!gsutil du -sh -a gs://msca-bdp-students-bucket/shared_data/bhadri/covid_tweets_df.parquet/

^C
Caught CTRL-C (signal 2) - exiting


In [10]:
#Read

covid_tweets_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/covid_tweets_df.parquet/")

22/03/18 00:12:29 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
22/03/18 00:12:35 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Stats

In [8]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

617.8 G  617.8 G  gs://msca-bdp-tweets/final_project



In [ ]:
%%time

tweets_df.count()

CPU times: user 1.13 s, sys: 335 ms, total: 1.46 s
Wall time: 4min 46s


100011000

## Twr_df

In [8]:
bucket_write = 'msca-bdp-students-bucket/shared_data/bhadri/tweets'

In [21]:
file1_path = 'gs://msca-bdp-students-bucket/shared_data/bhadri/tweets50k.csv'
#load csv to df
twr50k_df = spark.read.csv(file1_path, header='true', inferSchema='true', sep=',', quote='"')
# twr50k_df.cache()

In [22]:
twr50k_df.limit(3)

_c0,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
0,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,null,0,False,low,null,1482818815730008064,1482818815730008064,null,null,null,null,null,False,en,null,False,0,null,null,null,null,0,0,False,"""Row(contributors...",text='COVID 🦠 A...,truncated=False,user=Row(contrib...,created_at='Thu ...,default_profile=...,default_profile_...
1,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,"""Row(display_text...",0,False,low,null,1482818815960526850,1482818815960526850,null,null,null,null,null,False,en,null,False,0,null,null,null,null,0,0,False,null,"""<a href=""""https:...",Maryland Gov. Hog...,1642366529823,True,Row(contributors_...,null
2,null,null,Sun Jan 16 20:55:...,null,"Row(hashtags=[], ...",null,"""Row(display_text...",at least in NJ. ...,000 at the peak to 2,400 yesterday. \...,0,False,low,null,1482818816027541510,1482818816027541510,null,null,null,null,null,True,en,null,null,0,"""Row(contributors...",text='Do the Omi...,truncated=False,user=Row(contrib...,created_at='Sat ...,default_profile=...,default_profile_...,description='44 ...,Bears,Bulls,Black Hawks


In [30]:
testdf = twr50k_df.filter(twr50k_df.retweeted_status.contains('covid')).toPandas()


In [31]:
testdf.to_csv('gs://msca-bdp-students-bucket/shared_data/bhadri/testdf2.csv')

In [28]:
sample_df.select('lang').distinct().show()

+----+
|lang|
+----+
|  en|
+----+



In [48]:
print(twr50k_df.schema)

StructType(List(StructField(_c0,StringType,true),StructField(contributors,StringType,true),StructField(coordinates,StringType,true),StructField(created_at,StringType,true),StructField(display_text_range,StringType,true),StructField(entities,StringType,true),StructField(extended_entities,StringType,true),StructField(extended_tweet,StringType,true),StructField(favorite_count,StringType,true),StructField(favorited,StringType,true),StructField(filter_level,StringType,true),StructField(geo,StringType,true),StructField(id,StringType,true),StructField(id_str,StringType,true),StructField(in_reply_to_screen_name,StringType,true),StructField(in_reply_to_status_id,StringType,true),StructField(in_reply_to_status_id_str,StringType,true),StructField(in_reply_to_user_id,StringType,true),StructField(in_reply_to_user_id_str,StringType,true),StructField(is_quote_status,StringType,true),StructField(lang,StringType,true),StructField(place,StringType,true),StructField(possibly_sensitive,StringType,true),St

In [34]:
twr50k_df.columns

['_c0',
 'contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user',
 'withheld_in_countries']

In [20]:
#null in imp cols
col_list = ['user','text']
col_list2 = ['text']

In [59]:
from pyspark.sql.functions import col,isnan, when, count
twr50k_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in col_list]).show()

+-----+-----+
| user| text|
+-----+-----+
|15088|13832|
+-----+-----+



In [23]:
from pyspark.sql.functions import col,isnan, when, count
covid_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+----+----+
|user|text|
+----+----+
|   0|   0|
+----+----+



In [7]:
# covid_df = covid_df.withColumn("created_at_temp", covid_df.created_at.substr(5,30))

In [59]:
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
# covid_df = covid_df.withColumn("created_at_new", to_date("created_at","EEE MMM DD HH:mm:ss ZZ yyyy"))

In [93]:
covid_df = covid_df.drop("created_at_new","created_at_month","created_at_dayofmonth")

In [60]:
# covid_df = covid_df.withColumn("created_at_year", year(covid_df.created_at_new))
# covid_df = covid_df.withColumn("created_at_month", month(covid_df.created_at_new))
# covid_df = covid_df.withColumn("created_at_dayofmonth", dayofmonth(covid_df.created_at_new))
# covid_df.limit(5)

In [15]:
col_list = ['created_at_year']
covid_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+----------+
|created_at|
+----------+
|         0|
+----------+



In [26]:
col_list = ['created_at_temp']
covid_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+---------------+
|created_at_temp|
+---------------+
|              0|
+---------------+



NameError: name 'covid_df' is not defined

In [94]:
covid_df.limit(5)

id,id_str,user,place,text,extended_tweet,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,created_at_year,created_at_mon,created_at_day
1465067953163055105,1465067953163055105,"{false, Fri Nov 1...",null,RT @GregAbbott_TX...,null,"{[], null, [], []...",Sun Nov 28 21:19:...,null,0,0,0,0,false,false,en,2021,11,28
1465067953964163075,1465067953964163075,"{false, Sat Jan 1...",null,RT @SunScotNation...,null,"{[], null, [], [{...",Sun Nov 28 21:19:...,null,0,0,0,0,false,false,en,2021,11,28
1465067954027085831,1465067954027085831,"{false, Mon Sep 1...",null,RT @Covid19Critic...,null,"{[], null, [], []...",Sun Nov 28 21:19:...,null,0,0,0,0,false,false,en,2021,11,28
1465067954547007495,1465067954547007495,"{false, Thu Dec 1...",null,RT @MeidasTouch: ...,null,"{[], null, [], []...",Sun Nov 28 21:19:...,null,0,0,0,0,true,false,en,2021,11,28
1465067954685554688,1465067954685554688,"{false, Mon Jan 1...",null,RT @catturd2: The...,null,"{[], null, [], []...",Sun Nov 28 21:19:...,null,0,0,0,0,false,false,en,2021,11,28


In [ ]:
covid_df.filter(covid_df.created_at_year.isNull()).limit(10)

22/03/17 22:19:50 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 65 for reason Container marked as failed: container_1647546496475_0001_01_000066 on host: hub-msca-bdp-dphub-students-backup-bhadri-sw-9l8f.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/03/17 22:19:50 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 65 on hub-msca-bdp-dphub-students-backup-bhadri-sw-9l8f.c.msca-bdp-students.internal: Container marked as failed: container_1647546496475_0001_01_000066 on host: hub-msca-bdp-dphub-students-backup-bhadri-sw-9l8f.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.


id,id_str,user,place,text,extended_tweet,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,created_at_new,created_at_year,created_at_month,created_at_dayofmonth,created_at_mon,created_at_day


In [11]:
col_list = ['place','coordinates']
covid_tweets_df.select([count(when(col(c).isNull(), c)).alias(c) for c in col_list]).show()

+--------+-----------+
|   place|coordinates|
+--------+-----------+
|61430539|   61710934|
+--------+-----------+



In [21]:
test_df = covid_df.filter(covid_df.created_at_year.isNull()).limit(10000)
test_df.write.mode("overwrite").parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/test_date_df.parquet")

In [22]:
test_date_pq_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/test_date_df.parquet")

In [89]:
test_date_pq_df.limit(5)

id,id_str,user,place,text,extended_tweet,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang
1467276260103630850,1467276260103630850,"{false, Wed Jun 0...",null,RT @TheKaranMenon...,null,"{[], null, [], []...",Sat Dec 04 23:34:...,null,0,0,0,0,false,false,en
1467276260736782338,1467276260736782338,"{false, Wed Jun 0...",null,RT @WORLDMUSICAWA...,null,"{[{[21, 37], YGEn...",Sat Dec 04 23:34:...,null,0,0,0,0,false,false,en
1467276260598554631,1467276260598554631,"{false, Thu Oct 1...",null,RT @TroubledSamm:...,null,"{[], null, [], []...",Sat Dec 04 23:34:...,null,0,0,0,0,true,false,en
1467276260787515393,1467276260787515393,"{false, Thu Jul 1...",null,RT @disclosetv: L...,null,"{[{[128, 136], CO...",Sat Dec 04 23:34:...,null,0,0,0,0,false,false,en
1467276261525372929,1467276261525372929,"{false, Tue Oct 0...",null,"RT @MackayIM: ""Qu...",null,"{[], null, [], [{...",Sat Dec 04 23:34:...,null,0,0,0,0,false,false,en


In [88]:
test_date_pq_df = test_date_pq_df.drop("created_at_temp","created_at_mon_R","created_at_yr_R","created_at_day_R","created_at_mon_R2")

In [54]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
test_date_pq_df = test_date_pq_df.withColumn("created_at_new", to_date("created_at","EEE MMM dd HH:mm:ss ZZ yyyy"))

In [55]:
test_date_pq_df = test_date_pq_df.withColumn("created_at_year", year(test_date_pq_df.created_at_new))
test_date_pq_df = test_date_pq_df.withColumn("created_at_month", month(test_date_pq_df.created_at_new))
test_date_pq_df = test_date_pq_df.withColumn("created_at_dayofmonth", dayofmonth(test_date_pq_df.created_at_new))

In [86]:
test_date_pq_df = test_date_pq_df.withColumn("created_at_mon_R", month(to_date(trim(regexp_extract("created_at","\s\w+",0)),"MMM")))

In [78]:
test_date_pq_df = test_date_pq_df.withColumn("created_at_yr_R", regexp_extract("created_at","20\d\d$",0))

In [80]:
test_date_pq_df = test_date_pq_df.withColumn("created_at_day_R", trim(regexp_extract("created_at","\s\d\d\s",0)))

In [ ]:
covid_df.groupBy('created_at_year','created_at_month') \
        .agg(count(covid_df.id))

22/03/17 18:26:47 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1647536877841_0001_01_000050 on host: hub-msca-bdp-dphub-students-bhadri-sw-rfxn.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-03-17 18:26:47.240]Container killed on request. Exit code is 143
[2022-03-17 18:26:47.241]Container exited with a non-zero exit code 143. 
[2022-03-17 18:26:47.241]Killed by external signal
.
22/03/17 18:26:47 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1647536877841_0001_01_000048 on host: hub-msca-bdp-dphub-students-bhadri-sw-rfxn.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-03-17 18:26:47.241]Container killed on request. Exit code is 143
[2022-03-17 18:26:47.241]Container exited with a non-zero exit code 143. 
[2022-03-17 18:26:47.246]Killed by external signal
.
22/03/17 18:26:47 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting

created_at_year,created_at_month,count(id)
null,null,41163506
2022,1,20555993


In [ ]:
from pyspark.sql.functions import col,isnan, when, count
covid_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in col_list]).show()

In [16]:
corona_df = sample_df.filter(sql_fun.lower(sample_df.text).contains('corona'))
corona_df.count()

806681

In [18]:
# corona_df.write.mode("overwrite").parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet")

In [19]:
!gsutil du -sh -a gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet

1338225028   gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet


## Read Parquet Corona_pq

In [12]:
corona_pq_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/bhadri/corona_df.parquet")

In [10]:
corona_pq_df = corona_pq_df.withColumn("created_at_year", year(corona_pq_df.created_at_new))
corona_pq_df = corona_pq_df.withColumn("created_at_month", month(corona_pq_df.created_at_new))
corona_pq_df.limit(5)

id,id_str,user,text,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,created_at_new,year,month
1481853331467567104,1481853331467567104,"{false, Fri Dec 1...",When did we switc...,"{[], null, [], [{...",Fri Jan 14 04:59:...,null,0,0,0,0,false,false,en,2022-01-14,2022,1
1481853332931203072,1481853332931203072,"{false, Mon Sep 0...",RT @TruthGundlach...,"{[], null, [], []...",Fri Jan 14 04:59:...,null,0,0,0,0,false,false,en,2022-01-14,2022,1
1481853344855511042,1481853344855511042,"{false, Sat Jun 2...",RT @VigilantFox: ...,"{[], null, [], []...",Fri Jan 14 04:59:...,null,0,0,0,0,false,false,en,2022-01-14,2022,1
1481853369740410882,1481853369740410882,"{false, Mon Jul 2...",RT @InternetHippo...,"{[], null, [], []...",Fri Jan 14 04:59:...,null,0,0,0,0,true,false,en,2022-01-14,2022,1
1481853374861479939,1481853374861479939,"{false, Sat Mar 2...","This! ""42% of eli...","{[{[71, 89], Indi...",Fri Jan 14 04:59:...,null,0,0,0,0,false,false,en,2022-01-14,2022,1


In [9]:
corona_pq_df = corona_pq_df.withColumn("created_at_new", to_date(corona_pq_df.created_at.substr(5,30),"MMM DD HH:mm:ss xx yyyy"))

corona_pq_df.groupBy(year("created_at_new"),month("created_at_new")) \
        .agg(count(corona_pq_df.id))

year(created_at_new),month(created_at_new),count(id)
null,null,579768
2022,1,226913


In [27]:
corona_pq_df.count()

806681

In [77]:
corona_pq_df.limit(10)

id,id_str,user,text,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,type
1459541693749989386,1459541693749989386,"{false, Sun Apr 2...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541695960436740,1459541695960436740,"{false, Fri Aug 1...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541714323091458,1459541714323091458,"{false, Mon Feb 0...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541720274804741,1459541720274804741,"{false, Wed Nov 1...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541727757443073,1459541727757443073,"{false, Thu Nov 0...",RT @iran_policy: ...,"{[{[104, 109], Ir...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541750616309764,1459541750616309764,"{false, Mon Jun 1...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541767619940357,1459541767619940357,"{false, Sun Jan 0...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541767813013504,1459541767813013504,"{false, Fri Nov 2...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:20:...,null,0,0,0,0,false,false,en,tweet
1459541823995760642,1459541823995760642,"{false, Sat May 1...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:21:...,null,0,0,0,0,false,false,en,tweet
1459541848171716609,1459541848171716609,"{false, Wed Mar 1...",RT @DrEricDing: ?...,"{[], null, [], []...",Sat Nov 13 15:21:...,null,0,0,0,0,false,false,en,tweet


In [22]:
from pyspark.sql.functions import to_date
twr50k_df = twr50k_df.withColumn("created_at_new", twr50k_df.created_at.substr(5,30))

In [24]:
twr50k_df = twr50k_df.withColumn("created_at_new", to_date("created_at_new", "MMM DD HH:mm:ss xx yyyy"))

In [12]:
test1_pdf = covid_df.sample(0.001,42)

In [15]:
corona_pq_df = corona_pq_df.withColumn('user.name',col('user.name')) \
                .withColumn('user.scrname',col('user.screen_name')) \
                .withColumn('user.desc',col('user.description')) \
                .withColumn('user.id',col('user.id')) \
                .withColumn('user.loc',col('user.location')) \
                .withColumn('user.url',col('user.url'))

In [16]:
corona_pq_df.limit(10)

id,id_str,user,text,entities,created_at,coordinates,favorite_count,quote_count,retweet_count,reply_count,is_quote_status,retweeted,lang,user.name,user.scrname,user.desc,user.id,user.loc,user.tzone
1463574376809439234,1463574376809439234,"{false, Sat May 0...",RT @MetroUK: Seve...,"{[], null, [], []...",Wed Nov 24 18:24:...,null,0,0,0,0,false,false,en,Keith Webster,Keiths4pennorth,General interest ...,1391101501188288517,"Leeds, England",null
1463574396195332097,1463574396195332097,"{false, Wed Apr 2...",Coronavirus infec...,"{[], null, [], [{...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,Hindustan Times,htTweets,One of India's la...,36327407,India,null
1463574418962018306,1463574418962018306,"{false, Fri Mar 1...",RT @htTweets: Cor...,"{[], null, [], []...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,Corona Update Bot,CoronaUpdateBot,@plutonicindia ma...,1238467680773894146,null,null
1463574434258821125,1463574434258821125,"{false, Tue Mar 1...","RT @Tim_Roehn: ""I...","{[], null, [], []...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,Hans 🟢,Darioooooo23,null,3093509801,Deutschland,null
1463574442227994625,1463574442227994625,"{false, Tue Mar 0...",RT @DailyMail: Aa...,"{[], null, [], []...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,CovfefeForT,MeBeHealthy,Independent. Free...,516335684,"Massachusetts, USA",null
1463574490043006993,1463574490043006993,"{false, Tue Apr 1...",RT @Zobo_mx: due ...,"{[{[20, 28], COVI...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,Morri Colson,ColsonMorri,"God 🙏🏾 , Family...",983702386022764545,"Ocilla, GA",null
1463574608058241027,1463574608058241027,"{false, Fri Dec 0...",RT @drafzalniaz3:...,"{[], null, [], []...",Wed Nov 24 18:25:...,null,0,0,0,0,false,false,en,Kropstar,Krop_star,null,939209455958482944,null,null
1463574647128174602,1463574647128174602,"{false, Sat Oct 3...",RT @Aurea_MM: #CO...,"{[{[14, 22], COVI...",Wed Nov 24 18:26:...,null,0,0,0,0,false,false,en,Hugo Vásquez,hugovasquezv,Médico salubrista...,86438640,"Barcelona, Cataluña",null
1463574654250016773,1463574654250016773,"{false, Sun Feb 1...",RT @ylenews: #Cor...,"{[{[13, 25], Coro...",Wed Nov 24 18:26:...,null,0,0,0,0,false,false,en,Metinee Buaphan🇹...,metinee_buaphan,The more you read...,2347099007,"Tampere, Finland",null
1464253476091871241,1464253476091871241,"{false, Mon Sep 2...",RT @nichcarlson: ...,"{[], null, [], []...",Fri Nov 26 15:23:...,null,0,0,0,0,false,false,en,Kate Duffy,kate__duffy,Junior Business R...,1176202838382403586,"London, England",null


In [1]:
testdf = corona_pq_df.filter(sql_fun.lower(corona_pq_df.user.description).contains('student'))

NameError: name 'corona_pq_df' is not defined